<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loss-and-Metrics" data-toc-modified-id="Loss-and-Metrics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loss and Metrics</a></span></li><li><span><a href="#Camargo" data-toc-modified-id="Camargo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Camargo</a></span><ul class="toc-item"><li><span><a href="#Spezialized" data-toc-modified-id="Spezialized-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Spezialized</a></span></li><li><span><a href="#Concat" data-toc-modified-id="Concat-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Concat</a></span></li><li><span><a href="#Full_Concat" data-toc-modified-id="Full_Concat-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Full_Concat</a></span></li><li><span><a href="#PPMS" data-toc-modified-id="PPMS-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>PPMS</a></span></li></ul></li><li><span><a href="#Evermann" data-toc-modified-id="Evermann-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evermann</a></span><ul class="toc-item"><li><span><a href="#PPM" data-toc-modified-id="PPM-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>PPM</a></span></li></ul></li><li><span><a href="#Tax" data-toc-modified-id="Tax-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tax</a></span><ul class="toc-item"><li><span><a href="#Spezialized" data-toc-modified-id="Spezialized-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Spezialized</a></span></li><li><span><a href="#Shared" data-toc-modified-id="Shared-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Shared</a></span></li><li><span><a href="#Mixed" data-toc-modified-id="Mixed-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Mixed</a></span></li><li><span><a href="#PPM" data-toc-modified-id="PPM-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>PPM</a></span></li></ul></li><li><span><a href="#Mida" data-toc-modified-id="Mida-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Mida</a></span><ul class="toc-item"><li><span><a href="#PPM" data-toc-modified-id="PPM-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>PPM</a></span></li></ul></li></ul></div>

In [1]:
# default_exp baselines

Baseline implementations
===


This notebook introduces a few loss and metric functions. Afterwards, eight models based on four papers are re-implemented in `pytorch`. For each model, a `PPModel` is created.

In [2]:
#hide

%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

%matplotlib inline

In [3]:
#export
from mppn.imports import *
from mppn.preprocessing import *
from mppn.pipeline import *

In [4]:
# a flag, that controls if the training process is executed
_RUN_TRAINING=False

## Loss and Metrics

This section defines some metrics and loss functions. Apart from that, we use the standart loss functions and metrics from fastai and pytorch, namely accuracy, mae, and cross_entropy

In [5]:
#export
def maeDurDaysNormalize(p,yb,mean=0,std=0,unit=60*60*24):
    """
    Decodes time and converts from seconds to days
    Returns mae
    """
    p=p*std+mean
    yb=yb*std+mean
    return mae(p,yb)/(unit)

In [6]:
#export
def maeDurDaysMinMax(p,yb,minn=0,maxx=0,unit=60*60*24):
    """
    Decodes time and converts from seconds to days
    Returns mae
    """

    p=p*(maxx-minn) + minn
    yb=yb*(maxx-minn) + minn
    return mae(p,yb)/(unit)

In [7]:
#export
def _accuracy_idx(a,b,i): return accuracy(listify(a)[i],listify(b)[i])

In [8]:
#export
class AvgMetric(Metric):
    "Average the values of `func` taking into account potential different batch sizes"
    def __init__(self, func):  self.func = func
    def reset(self):           self.total,self.count = 0.,0
    def accumulate(self, learn):
        bs = find_bs(learn.yb)
        self.total += learn.to_detach(self.func(learn.pred, *learn.yb))*bs
        self.count += bs
    @property
    def value(self): return self.total/self.count if self.count != 0 else None
    @property
    def name(self):
        return self.func.__name__ if hasattr(self.func, '__name__') else self.func.func.__name__

In [9]:
#export
def get_metrics(o,date_col='timestamp_Relative_elapsed'):
    'A utility function that automatically selects the correct metric functions based on the PPObj o'
    number_cats=len(o.ycat_names)

    accuracies=[]
    for i in range(number_cats):
        accuracy_func=partial(_accuracy_idx,i=i)
        accuracy_func.__name__= f"acc_{o.ycat_names[i]}"
        accuracy_func=AvgMetric(accuracy_func)
        accuracies.append(accuracy_func)
    mae_days=None
    if len(o.ycont_names)>0:
        if 'minmax' in o.ycont_names[0]: # Here we expect only one timestamp
            minn,maxx = (o.procs.min_max.mins[date_col],
                         o.procs.min_max.maxs[date_col])
            mae_days=lambda p,y: maeDurDaysMinMax(listify(p)[-1],listify(y)[-1],minn=minn,maxx=maxx)
        else:
            mean,std=(o.procs.normalize.means[date_col],
                      o.procs.normalize.stds[date_col])
            mae_days=lambda p,y: maeDurDaysNormalize(listify(p)[-1],listify(y)[-1],mean=mean,std=std)
        mae_days.__name__='mae_days'
    return L(accuracies)+mae_days

In [11]:
#export
def multi_loss_sum(o,p,y):
    '''Multi Loss function that sums up multiple loss functions. 
    The selection of the loss function is based on the PPObj o'''
    p,y=listify(p),listify(y)
    len_cat,len_cont=len(o.ycat_names),len(o.ycont_names)
    cross_entropies=[F.cross_entropy(p[i],y[i]) for i in range(len_cat)]
    maes=[mae(p[i],y[i]) for i in range(len_cat,len_cat+len_cont)]
    return torch.sum(torch.stack(list(L(cross_entropies)+L(maes))))

## Camargo

**Input**: activity, resource, duration  
**Output**: activity, resource, duration  
**Loss**: sum(cross_entropy(activity),cross_entropy(resource),mae(duration))

In [12]:
log=import_log(EventLogs.Helpdesk)
o=PPObj(log,[Categorify,Datetify,Normalize()],date_names=['timestamp'],cat_names=['activity','resource'],
    y_names=['activity','resource','timestamp_Relative_elapsed'],splits=split_traces(log))
o
dls=o.get_dls()

In [13]:
xcat,xcont,y=dls.one_batch()

In [14]:
xcat.shape,xcont.shape,len(y)

(torch.Size([64, 2, 64]), torch.Size([64, 64]), 3)

### Spezialized

In [15]:
#export
class Camargo_specialized(torch.nn.Module) :
    def __init__(self, o) :
        super().__init__()
        hidden=25
        vocab_act=len(o.procs.categorify['activity'])
        vocab_res=len(o.procs.categorify['resource'])
        emb_dim_act = int(sqrt(vocab_act))+1
        emb_dim_res = int(sqrt(vocab_res))+1

        self.emb_act = nn.Embedding(vocab_act,emb_dim_act)
        self.emb_res = nn.Embedding(vocab_res,emb_dim_res)

        self.lstm_act = nn.LSTM(emb_dim_act, hidden, batch_first=True, num_layers=2)
        self.lstm_res = nn.LSTM(emb_dim_res, hidden, batch_first=True, num_layers=2)
        self.lstm_tim = nn.LSTM(1, hidden, batch_first=True, num_layers=2)

        self.linear_act = nn.Linear(hidden, vocab_act)
        self.linear_res = nn.Linear(hidden, vocab_res)
        self.linear_tim = nn.Linear(hidden, 1)
    def forward(self, xcat,xcont):
        x_act,x_res,x_tim=xcat[:,0],xcat[:,1],xcont[:,:,None]
        x_act = self.emb_act(x_act)
        x_act,_ = self.lstm_act(x_act)
        x_act = x_act[:,-1]
        x_act = self.linear_act(x_act)
        x_act = F.softmax(x_act,dim=1)

        x_res = self.emb_res(x_res)
        x_res,_ = self.lstm_res(x_res)
        x_res = x_res[:,-1]
        x_res = self.linear_res(x_res)
        x_res = F.softmax(x_res,dim=1)

        x_tim,_ = self.lstm_tim(x_tim)
        x_tim = x_tim[:,-1]
        x_tim = self.linear_tim(x_tim)
        return x_act,x_res,x_tim

In [16]:
m=Camargo_specialized(o)

In [17]:
p=m(xcat,xcont)

In [18]:
tuple(i.shape for i in p)

(torch.Size([64, 15]), torch.Size([64, 23]), torch.Size([64, 1]))

In [19]:
%%time
if _RUN_TRAINING:
    log=import_log(EventLogs.Helpdesk)
    o=PPObj(log,[Categorify,Datetify,Normalize],date_names=['timestamp'],cat_names=['activity','resource'],
        y_names=['activity','resource','timestamp_Relative_elapsed'],splits=split_traces(log),)
    dls=o.get_dls()
    m=Camargo_specialized(o)
    loss=0
    metrics=get_metrics(o)
    loss=partial(multi_loss_sum,o)
    train_validate(dls,m,loss=loss,metrics=metrics,epoch=1,print_output=True,output_index=[1,2,3])

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.81 µs


### Concat

In [20]:
#export
class Camargo_concat(torch.nn.Module) :
    def __init__(self, o ) :
        super().__init__()
        hidden=25
        vocab_act=len(o.procs.categorify['activity'])
        vocab_res=len(o.procs.categorify['resource'])
        emb_dim_act = int(sqrt(vocab_act))+1
        emb_dim_res = int(sqrt(vocab_res))+1

        self.emb_act = nn.Embedding(vocab_act,emb_dim_act)
        self.emb_res = nn.Embedding(vocab_res,emb_dim_res)

        self.lstm_concat= nn.LSTM(emb_dim_act+emb_dim_res, hidden, batch_first=True, num_layers=1)
        self.lstm_act = nn.LSTM(hidden, hidden, batch_first=True, num_layers=1)
        self.lstm_res = nn.LSTM(hidden, hidden, batch_first=True, num_layers=1)
        self.lstm_tim = nn.LSTM(1, hidden, batch_first=True, num_layers=2)

        self.linear_act = nn.Linear(hidden, vocab_act)
        self.linear_res = nn.Linear(hidden, vocab_res)
        self.linear_tim = nn.Linear(hidden, 1)
    def forward(self, xcat,xcont):
        x_act,x_res,x_tim=xcat[:,0],xcat[:,1],xcont[:,:,None]
        x_act = self.emb_act(x_act)

        x_res = self.emb_res(x_res)
        x_concat=torch.cat((x_act, x_res), 2)
        x_concat,_=self.lstm_concat(x_concat)

        x_act,_ = self.lstm_act(x_concat)
        x_act = x_act[:,-1]
        x_act = self.linear_act(x_act)
        x_act = F.softmax(x_act,dim=1)

        x_res,_ = self.lstm_res(x_concat)
        x_res = x_res[:,-1]
        x_res = self.linear_res(x_res)
        x_res = F.softmax(x_res,dim=1)

        x_tim,_ = self.lstm_tim(x_tim)
        x_tim = x_tim[:,-1]
        x_tim = self.linear_tim(x_tim)
        return x_act,x_res,x_tim

In [21]:
m=Camargo_concat(o)
p=m(xcat,xcont)

In [22]:
%%time
if _RUN_TRAINING:

    log=import_log(EventLogs.Helpdesk)
    o=PPObj(log,[Categorify,Datetify,Normalize()],date_names=['timestamp'],cat_names=['activity','resource'],
        y_names=['activity','resource','timestamp_Relative_elapsed'],splits=split_traces(log),)
    dls=o.get_dls()
    m=Camargo_concat(o)
    loss=0
    loss=partial(multi_loss_sum,o)
    train_validate(dls,m,loss=loss,metrics=get_metrics(o),epoch=1,print_output=True,output_index=[1,2,3])

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs


### Full_Concat 

In [23]:
#export
class Camargo_fullconcat(torch.nn.Module) :


    def __init__(self, o  ) :
        super().__init__()

        hidden=25
        vocab_act=len(o.procs.categorify['activity'])
        vocab_res=len(o.procs.categorify['resource'])
        emb_dim_act = int(sqrt(vocab_act))+1
        emb_dim_res = int(sqrt(vocab_res))+1

        self.emb_act = nn.Embedding(vocab_act,emb_dim_act)
        self.emb_res = nn.Embedding(vocab_res,emb_dim_res)

        self.lstm_concat= nn.LSTM(emb_dim_act+emb_dim_res+1, hidden, batch_first=True, num_layers=1)
        self.lstm_act = nn.LSTM(hidden, hidden, batch_first=True, num_layers=1)
        self.lstm_res = nn.LSTM(hidden, hidden, batch_first=True, num_layers=1)
        self.lstm_tim = nn.LSTM(hidden, hidden, batch_first=True, num_layers=1)

        self.linear_act = nn.Linear(hidden, vocab_act)
        self.linear_res = nn.Linear(hidden, vocab_res)
        self.linear_tim = nn.Linear(hidden, 1)
    def forward(self, xcat,xcont):
        x_act,x_res,x_tim=xcat[:,0],xcat[:,1],xcont[:,:,None]
        x_act = self.emb_act(x_act)

        x_res = self.emb_res(x_res)
        x_concat=torch.cat((x_act, x_res,x_tim), 2)
        x_concat,_=self.lstm_concat(x_concat)

        x_act,_ = self.lstm_act(x_concat)
        x_act = x_act[:,-1]
        x_act = self.linear_act(x_act)
        x_act = F.softmax(x_act,dim=1)

        x_res,_ = self.lstm_res(x_concat)
        x_res = x_res[:,-1]
        x_res = self.linear_res(x_res)
        x_res = F.softmax(x_res,dim=1)

        x_tim,_ = self.lstm_tim(x_concat)
        x_tim = x_tim[:,-1]
        x_tim = self.linear_tim(x_tim)
        return x_act,x_res,x_tim

In [24]:
m=Camargo_fullconcat(o)
p=m(xcat,xcont)

In [25]:
%%time
if  _RUN_TRAINING:
    log=import_log(EventLogs.Helpdesk)
    o=PPObj(log,[Categorify,Datetify,Normalize],date_names=['timestamp'],cat_names=['activity','resource'],
        y_names=['activity','resource','timestamp_Relative_elapsed'],splits=split_traces(log),)
    dls=o.get_dls()
    m=Camargo_fullconcat(o)
    loss=0
    loss=partial(multi_loss_sum,o)
    train_validate(dls,m,loss=loss,metrics=get_metrics(o),epoch=1,print_output=True,output_index=[1,2,3])

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 7.39 µs


### PPMS

In [26]:
#export
class PPM_Camargo_Spezialized(PPModel):

    model = Camargo_specialized
    date_names=['timestamp']
    cat_names=['activity','resource']
    y_names=['activity','resource','timestamp_Relative_elapsed']
    cont_names=None
    procs=[Categorify,Datetify,Normalize,FillMissing]

    def setup(self):
        o=PPObj(self.log,self.procs,cat_names=self.cat_names,date_names=self.date_names,y_names=self.y_names,
                cont_names=self.cont_names,splits=self.splits)

        loss=partial(multi_loss_sum,o)

        # Next event prediction training
        print('Next event prediction training')
        dls=o.get_dls(bs=self.bs)
        m=self.model(o)
        self.nsp,self.nrp,self.dtnp=self._train_validate(dls,m,loss=loss,metrics=get_metrics(o),
                                                   output_index=[1,2,3])


        # Last event prediction training
        print('Last event prediction training')
        dls=o.get_dls(outcome=True,bs=self.bs)
        m=self.model(o)
        self.op,self.lrp,self.dtlp=self._train_validate(dls,m,loss=loss,metrics=get_metrics(o),
                                                 output_index=[1,2,3])



    def next_step_prediction(self): return self.nsp

    def next_resource_prediction(self):return self.nrp

    def last_resource_prediction(self): return self.lrp
    def outcome_prediction(self): return self.op
    def duration_to_next_event_prediction(self): return self.dtnp
    def duration_to_end_prediction(self): return self.dtlp
    def activity_suffix_prediction(self): pass
    def resource_suffix_prediction(self): pass

In [27]:
#export
class PPM_Camargo_concat(PPM_Camargo_Spezialized):
    model = Camargo_concat

class PPM_Camargo_fullconcat(PPM_Camargo_Spezialized):
    model = Camargo_fullconcat

In [28]:
%%time
if _RUN_TRAINING:
    path=EventLogs.Helpdesk
    log=import_log(path)
    ppm=PPM_Camargo_fullconcat(log,get_ds_name(path),print_output=True,epoch=1,bs=512,
                               splits=split_traces(log))
    ppm.evaluate()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.05 µs


## Evermann

**Input**:  activity or resource  
**Output**: activity or resource  
**Loss**: cross_entropy(activity) or cross_entropy(resource)

In [29]:
#export

class Evermann(torch.nn.Module) :
    def __init__(self, o) :
        super().__init__()
        vocab_size=len(o.procs.categorify[o.y_names[0]])
        hidden_dim=125
        emb_dim = 5


        self.embeddings = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True, num_layers=2)
        self.linear = nn.Linear(hidden_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x.squeeze())
        x = self.dropout(x)
        x, _ = self.lstm(x)
        x = self.linear(x[:,-1])
        return F.softmax(x,dim=1)

In [30]:
col='activity'
o=PPObj(log,procs=[Categorify],cat_names=col,y_names=col,splits=split_traces(log))
dls=o.get_dls()
xb,yb=dls.one_batch()

In [31]:
xb.shape,yb.shape

(torch.Size([64, 64]), torch.Size([64]))

In [32]:
m=Evermann(o)

In [33]:
pb=m(xb)
F.cross_entropy(pb,yb),accuracy(pb,yb)

(tensor(2.7060, grad_fn=<NllLossBackward>), TensorBase(0.))

In [34]:
partial(multi_loss_sum,o)(pb,yb)

tensor(2.7060, grad_fn=<SumBackward0>)

In [35]:
loss=partial(multi_loss_sum,o)
metrics=get_metrics(o)

In [36]:
%%time
if _RUN_TRAINING:
    train_validate(dls,m,loss=loss,metrics=metrics,epoch=1)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


### PPM

In [37]:
#export
class PPM_Evermann(PPM_RNNwEmbedding): 
    model = Evermann

In [38]:
%%time
if _RUN_TRAINING:
    runner([EventLogs.Helpdesk,EventLogs.Mobis],[PPM_Evermann,PPM_RNNwEmbedding],
           bs=1024,epoch=1,print_output=False)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


## Tax

**Input**:  activity or duration  
**Output**: activity or duration  
**Loss**: cross_entropy(activity) or mae(duration) in days

### Spezialized

In [39]:
#export
class Tax_et_al_spezialized(torch.nn.Module) :
    def __init__(self,o) :
        super().__init__()
        vocab_size=len(o.procs.categorify[o.y_names[0]])
        hidden_dim=125
        self.lstm_act = nn.LSTM(vocab_size, hidden_dim, batch_first=True, num_layers=2)
        self.lstm_tim = nn.LSTM(3, hidden_dim, batch_first=True, num_layers=2)

        self.linear_act = nn.Linear(hidden_dim, vocab_size)
        self.linear_tim = nn.Linear(hidden_dim, 1)


    def forward(self, xcat,xcont):
        x_act,x_tim = xcat.permute(0,2,1),xcont.squeeze().permute(0,2,1)
        x_act, _ = self.lstm_act(x_act.float())
        x_act=self.linear_act(x_act[:,-1])
        x_act=F.softmax(x_act,dim=1)
        x_tim, _ = self.lstm_tim(x_tim)
        x_tim=self.linear_tim(x_tim[:,-1])
        return x_act,x_tim

In [40]:
log=import_log(EventLogs.BPIC_12)

In [41]:
datetify=Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed'])
o=PPObj(log,[Categorify,OneHot,datetify,Normalize],cat_names='activity',splits=split_traces(log),
        date_names='timestamp',y_names=['activity','timestamp_Relative_elapsed'])
dls=o.get_dls()

In [42]:
xcat,xcont,yb=(dls.one_batch())

In [43]:
o.cont_names

(#3) ['timestamp_secSinceSunNoon','timestamp_secSinceNoon','timestamp_Relative_elapsed']

In [44]:
m=Tax_et_al_spezialized(o)

In [45]:
p=m(xcat,xcont)

In [46]:
loss=partial(multi_loss_sum,o)
loss(p,yb)

TensorBase(4.5369, grad_fn=<AliasBackward>)

In [47]:
metrics=get_metrics(o)


In [48]:
if _RUN_TRAINING:
    datetify=Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed'])
    o=PPObj(log,[Categorify,OneHot,datetify,Normalize],cat_names='activity',splits=split_traces(log),
            date_names='timestamp',y_names=['activity','timestamp_Relative_elapsed'])
    dls=o.get_dls()
    m=Tax_et_al_spezialized(o)
    loss=partial(multi_loss_sum,o)
    metrics=get_metrics(o)
    train_validate(dls,m,loss=loss,metrics=metrics,epoch=1,output_index=[1,2])

### Shared

In [49]:
#export
class Tax_et_al_shared(torch.nn.Module) :
    def __init__(self,o) :
        super().__init__()
        vocab_size=len(o.procs.categorify[o.y_names[0]])
        hidden_dim=125
        self.lstm = nn.LSTM(vocab_size+3, hidden_dim, batch_first=True, num_layers=2)

        self.linear_act = nn.Linear(hidden_dim, vocab_size)
        self.linear_tim = nn.Linear(hidden_dim, 1)


    def forward(self,xcat,xcont):
        x_act,x_tim = xcat.permute(0,2,1),xcont.squeeze().permute(0,2,1)

        x_concat=torch.cat((x_act.float(), x_tim), 2)
        x_concat, _ = self.lstm(x_concat)

        x_act=self.linear_act(x_concat[:,-1])
        x_act=F.softmax(x_act,dim=1)

        x_tim=self.linear_tim(x_concat[:,-1])
        return x_act,x_tim

In [50]:
m=Tax_et_al_shared(o)

In [51]:
p=m(xcat,xcont)

In [52]:
loss(p,yb)

TensorBase(4.5371, grad_fn=<AliasBackward>)

In [53]:
if _RUN_TRAINING:
    datetify=Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed'])
    o=PPObj(log,[Categorify,OneHot,datetify,Normalize],cat_names='activity',splits=split_traces(log),
            date_names='timestamp',y_names=['activity','timestamp_Relative_elapsed'])
    m=Tax_et_al_shared(o)
    dls=o.get_dls()
    loss=partial(multi_loss_sum,o)
    metrics=get_metrics(o)
    train_validate(dls,m,loss=loss,metrics=metrics,epoch=1,output_index=[1,2])

### Mixed

In [54]:
#export
class Tax_et_al_mixed(torch.nn.Module) :
    def __init__(self,o,numlayers_shared=3,numlayers_single=3) :
        super().__init__()
        vocab_size=len(o.procs.categorify[o.y_names[0]])
        hidden_dim=125

        self.lstm_act = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers=numlayers_single)
        self.lstm_tim = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, num_layers=numlayers_single)
        self.lstm = nn.LSTM(vocab_size+3, hidden_dim, batch_first=True, num_layers=numlayers_shared)

        self.linear_act = nn.Linear(hidden_dim, vocab_size)
        self.linear_tim = nn.Linear(hidden_dim, 1)


    def forward(self,xcat,xcont):
        x_act,x_tim = xcat.permute(0,2,1),xcont.squeeze().permute(0,2,1)


        x_concat=torch.cat((x_act.float(), x_tim), 2)
        x_concat, _ = self.lstm(x_concat)

        x_act, _ = self.lstm_act(x_concat)
        x_act=self.linear_act(x_act[:,-1])
        x_act=F.softmax(x_act,dim=1)

        x_tim, _ = self.lstm_tim(x_concat)
        x_tim=self.linear_tim(x_tim[:,-1])
        return x_act,x_tim

In [55]:
m=Tax_et_al_mixed(o)

In [56]:
xcat,xcont,yb=dls.one_batch()

In [57]:
p=m(xcat,xcont)

In [58]:
loss(p,yb)

TensorBase(4.5081, grad_fn=<AliasBackward>)

In [59]:
log=import_log(EventLogs.BPIC_12)
traces=split_traces(log)[0][:100]
splits=traces[:60],traces[60:80],traces[80:100]
splits=split_traces(log)

In [60]:
if _RUN_TRAINING:
    datetify=Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed'])
    o=PPObj(log,[Categorify,OneHot,datetify,Normalize],cat_names='activity',splits=splits,
            date_names='timestamp',y_names=['activity','timestamp_Relative_elapsed'])
    m=Tax_et_al_mixed(o)  
    dls=o.get_dls()
    loss=partial(multi_loss_sum,o)
    metrics=get_metrics(o)
    train_validate(dls,m,loss=loss,metrics=metrics,epoch=1,output_index=[1,2],lr_find=False)

### PPM

In [61]:
#export
class PPM_Tax_Spezialized(PPModel):

    model = Tax_et_al_spezialized
    date_names=['timestamp']
    cat_names=['activity']
    y_names=['activity','timestamp_Relative_elapsed']
    cont_names=None
    procs=[Categorify,OneHot,Datetify(date_encodings=['secSinceSunNoon','secSinceNoon','Relative_elapsed']),
           Normalize,FillMissing]

    def setup(self):
        o=PPObj(self.log,self.procs,cat_names=self.cat_names,date_names=self.date_names,y_names=self.y_names,
                cont_names=self.cont_names,splits=self.splits)

        loss=partial(multi_loss_sum,o)

        # Next event prediction training
        print('Next event prediction training')
        dls=o.get_dls(bs=self.bs)
        m=self.model(o)
        self.nsp,self.dtnp=self._train_validate(dls,m,loss=loss,metrics=get_metrics(o),
                                                   output_index=[1,2])


        # Last event prediction training
        print('Last event prediction training')
        dls=o.get_dls(outcome=True,bs=self.bs)
        m=self.model(o)
        self.op,self.dtlp=self._train_validate(dls,m,loss=loss,metrics=get_metrics(o),
                                                 output_index=[1,2])



    def next_step_prediction(self): return self.nsp


    def outcome_prediction(self): return self.op
    def duration_to_next_event_prediction(self): return self.dtnp
    def duration_to_end_prediction(self): return self.dtlp
    def activity_suffix_prediction(self): pass
    def resource_suffix_prediction(self): pass

In [62]:
#export
class PPM_Tax_Shared(PPM_Tax_Spezialized):
    model = Tax_et_al_shared

class PPM_Tax_Mixed(PPM_Tax_Spezialized):
    model = Tax_et_al_mixed

In [63]:
%%time
if _RUN_TRAINING:
    log=import_log(path)
    ppm=PPM_Tax_Mixed(log,get_ds_name(path),sample=True,print_output=True,epoch=1,bs=512,splits=split_traces(log))
    ppm.evaluate()

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.29 µs


## Mida

**Input**: multi categorical atts and multi cont atts  
**Output**: activity or resource or duration  
**Loss**: cross_entropy(activity) or cross_entropy(resource) or mae(duration) in days

In [64]:
#export
class MiDA(Module):
    def __init__(self,o,seq_len=64) :
        super().__init__()
        hidden_dim1=100
        hidden_dim2=100

        out=o.y_names[0]
        emb_szs=[(len(o.procs.categorify[c]),len(o.procs.categorify[c])//2 ) for c in o.cat_names ]
        self.embeds = nn.ModuleList([Embedding(ni, nf) for ni,nf in emb_szs])
        self.n_cont=len(o.cont_names)
        self.n_emb = sum(e.embedding_dim for e in self.embeds)
        self.lstm1=nn.LSTM(self.n_cont+self.n_emb, hidden_dim1, batch_first=True, num_layers=1)

        self.bn_cont = nn.BatchNorm1d(self.n_cont)
        self.lstm2=nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True, num_layers=1)
        #self.bn2=nn.BatchNorm1d(seq_len)
        #self.bn1=nn.BatchNorm1d(seq_len)

        if out in  o.procs.categorify.classes:
            self.lin=nn.Linear(hidden_dim2,len(o.procs.categorify[out]))
            self.is_classifier=True
        else:
            self.lin=nn.Linear(hidden_dim2,1)
            self.is_classifier=False


    def forward(self, x_cat,x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embeds)]
            x = torch.cat(x, 2)
        if self.n_cont != 0:
            if self.n_cont == 1: x_cont=x_cont[:,None]
            if self.bn_cont: x_cont=self.bn_cont(x_cont).transpose(2,1)
            x = torch.cat([x, x_cont], 2) if self.n_emb != 0 else x_cont

        x,_=self.lstm1(x)
        #x= self.bn1(x)
        x,h=self.lstm2(x)
        #x=self.bn2(x[:,-1])
        x=self.lin(x[:,-1])
        if self.is_classifier: x=F.softmax(x,1)
        return x

In [65]:
log=import_log(EventLogs.Helpdesk)

In [66]:
o=PPObj(log,[Categorify,Normalize,Datetify,FillMissing],
        cat_names=['activity','resource'],date_names=['timestamp'],y_names='activity',splits=split_traces(log))
dls=o.get_dls()

In [67]:
xcat,xcont,yb=dls.one_batch()

In [68]:
m=MiDA(o)

In [69]:
p=m(xcat,xcont)

In [70]:
%%time
if  _RUN_TRAINING:
    log=import_log(EventLogs.BPIC_12)
    o=PPObj(log,[Categorify,Normalize,Datetify,FillMissing],
            cat_names=['activity','resource'],date_names=['timestamp'],
            splits=split_traces(log))
    o.set_y_names('timestamp_Relative_elapsed')
    dls=o.get_dls(bs=512)
    seq_len=o.items.event_id.max()
    m=MiDA(o,seq_len)
    loss=partial(multi_loss_sum,o)
    metrics=get_metrics(o)
    train_validate(dls,m,epoch=5,loss=loss,metrics=metrics,print_output=True)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.29 µs


### PPM

In [71]:
subsequences_fast??

In [72]:
#export

class PPM_MiDA(PPModel):
    model = MiDA

    procs=[Categorify,Normalize,Datetify,FillMissing]

    def _attr_from_dict(self,ds_name):
        if not self.attr_dict: raise AttributeError('attr_dict is required!')

        return (listify(self.attr_dict[self.ds_name]['cat attr']),
                listify(self.attr_dict[self.ds_name]['num attr']),
                listify(self.attr_dict[self.ds_name]['date attr']))

    def setup(self):
        cat_names,cont_names,date_names=self._attr_from_dict(self.ds_name)
        self.o=PPObj(self.log,[Categorify,Normalize,Datetify,FillMissing],
                     cat_names=cat_names,date_names=date_names,cont_names=cont_names,
                     splits=self.splits)





    def next_step_prediction(self,col='activity',outcome=False):
        seq_len=(self.o.items.event_id.max()) # seq_len = max trace len, Todo make it nicer
        self.o.set_y_names(col)
        print(self.o.y_names)
        dls=self.o.get_dls(bs=self.bs,outcome=outcome)
        m=self.model(self.o,seq_len)
        loss=partial(multi_loss_sum,self.o)
        metrics=get_metrics(self.o)
        return self._train_validate(dls,m,loss=loss,metrics=metrics)

    def next_resource_prediction(self):return self.next_step_prediction(outcome=False,col='resource')

    def last_resource_prediction(self): return self.next_step_prediction(outcome=True,col='resource')
    def outcome_prediction(self): return self.next_step_prediction(outcome=True,col='activity')

    def duration_to_next_event_prediction(self):
        return self.next_step_prediction(outcome=False,col='timestamp_Relative_elapsed')

    def duration_to_end_prediction(self):
        return self.next_step_prediction(outcome=True,col='timestamp_Relative_elapsed')

In [73]:
#export
def create_attr_dict(attr_list):
    attr_df=pd.DataFrame(attr_list,columns=['name','cat attr','num attr','date attr'])
    attr_df.index=attr_df.name
    attr_df.drop('name',axis=1,inplace=True)
    attr_df.index.name=""
    attr_dict=attr_df.apply(lambda x:(x.str.split(', '))).T.to_dict()
    return attr_dict

In [74]:
#export

attr_list=[
    ['BPIC12','activity, resource','AMOUNT_REQ','timestamp'],
    ['BPIC12_W','activity, resource','AMOUNT_REQ','timestamp'],
    ['BPIC12_Wc','activity, resource','AMOUNT_REQ','timestamp'],
    ['BPIC12_O','activity, resource','AMOUNT_REQ','timestamp'],
    ['BPIC12_A','activity, resource','AMOUNT_REQ','timestamp'],
    ['Mobis','activity, resource, type','cost','timestamp'],
    ['BPIC13_CP','activity, resource, resource country, organization country, organization involved, impact, product, org:role',
     None,'timestamp'],
    ['Helpdesk','activity, resource',None,'timestamp'],
    ['BPIC17_O','activity, Action, NumberOfTerms, resource',
     'FirstWithdrawalAmount, MonthlyCost, OfferedAmount, CreditScore', 'timestamp'],
    ['BPIC20_RFP','org:role, activity, resource, Project, Task, OrganizationalEntity',
     'RequestedAmount','timestamp']
]


attr_dict=create_attr_dict(attr_list)

In [75]:
%%time
if _RUN_TRAINING:
    path=EventLogs.Helpdesk
    log=import_log(path)
    ppm=PPM_MiDA(log,get_ds_name(path),print_output=True,epoch=1,bs=512,attr_dict=attr_dict,splits=split_traces(log))
    ppm.setup()
    ppm.next_step_prediction()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
